In [7]:
%pip install fastapi pydantic toml

Note: you may need to restart the kernel to use updated packages.


In [8]:
# from rs import skill_check

# def test_skill_check(metadata):
#     print(skill_check("sanity", None, metadata))

# metadata = {
#     "character": {"sanity":30},
#     "symptom":[
#         "incoherent speech",
#         "uncontrollable twitching, trembling",
#         "delusions of persecution",
#         "strange appetites (dirt, clay, etc...)",
#         "scratches, punches, and bruises all over the body"
#     ]
# }

# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)

# print(metadata["character"])


In [9]:
import re
import tomllib
import functools
from typing import Literal

import ipywidgets as widgets
from IPython.display import display

from dotenv import load_dotenv

load_dotenv()

from rs import Message, Config, chat, skill_check
import d20

action_pattern = re.compile(r"(\d+). *(.+)")
skill_pattern = re.compile(r"[\w_]+")
skill_difficulty_pattern = re.compile(r"(easy|medium|hard)")

def next_round(b):
    global current_round, initial_messages
    current_round += 1
    initial_messages = config.initial_messages(current_round, config.metadata)

    print(initial_messages[-1].content)
    display(get_controls(initial_messages[-1]))

def get_user_message_tail():
    count = 0
    for msg in initial_messages:
        if msg.role == "user":
            count += 1

    is_final =  ", FINAL ROUND" if  current_round == len(config.rounds) else ""
    
    # print(config.metadata["character"])
    
    if "tags" in config.metadata["character"] and "insane" in config.metadata["character"]["tags"]:
        return f"TURN: {count + 1} {is_final} (I'm insane!)"
    
    return f"TURN: {count + 1} {is_final}"
    

def do_chat(role:Literal["user", "system", "assistant"], content: str):
    print(f"{content}\n\n")

    initial_messages.append(Message(role=role, content=content))
    msg = chat(initial_messages, lambda delta: print(delta, end=""))

    initial_messages.append(msg)
    display(get_controls(msg))


def on_skill(b, skill:str, difficulty:str):
    print(config.metadata["character"])
    content = skill_check(skill, difficulty, config.metadata)
    content += f"\n{get_user_message_tail()}"
    do_chat("user", content)

def on_action(b, index:int):
    content = f"I select option {index}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_input(b, input:widgets.Text):
    content = f"{input.value}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_action(b):
    hbox = widgets.HBox(layout=widgets.Layout(width="100%"))
    action_text = widgets.Text(layout=widgets.Layout(width="100%"))
    submit = widgets.Button(description="SUBMIT", layout=widgets.Layout(width="fit-content"))
    submit.on_click(functools.partial(on_custom_input, input=action_text))
    hbox.children += (action_text, submit) 
    display(hbox)

def get_controls(msg:Message) -> widgets.Box:
    dict = msg.dict()
    buttons = []
    if "possible actions" in dict.keys():
        matches = action_pattern.finditer(dict["possible actions"])
        for match in matches:
            btn = widgets.Button(description=match.group(1), tooltip=match.group(2), layout=widgets.Layout(width="auto"))
            btn.on_click(functools.partial(on_action, index=int(match.group(1))))
            buttons.append(btn)
    elif "skill" in dict.keys() and "difficulty" in dict.keys():
        match = skill_pattern.match(dict["skill"])
        skill = match.group(0)
        match = skill_difficulty_pattern.match(dict["difficulty"])
        difficulty = match.group(0)
        skill_button = widgets.Button(description=f"{skill.upper()} [{difficulty.upper()}]", layout=widgets.Layout(width="auto"))
        skill_button.on_click(functools.partial(on_skill, skill=skill, difficulty=difficulty))
        buttons.append(skill_button)
    elif "summary" in dict.keys():
        # add summary as a memory to the metadata
        # it will be used in the next round.
        config.metadata["memories"] = []
        config.metadata["memories"].append(dict["summary"])
        next_btn = widgets.Button(description="NEXT ROUND", layout=widgets.Layout(width="auto"))
        next_btn.on_click(next_round)
        buttons.append(next_btn)
    
    custom_action_button = widgets.Button(description="CUSTOM")
    custom_action_button.on_click(on_custom_action)
    buttons.append(custom_action_button)

    return widgets.HBox(buttons)

# open config file
config_name = "data/config_skillcheck_test.toml"
config: Config | None = None
with open(config_name, "rb") as f:
    data = tomllib.load(f)
    config = Config(**data)

current_round = 0
initial_messages = config.initial_messages(current_round, config.metadata)

print(initial_messages[-1].content)
display(get_controls(initial_messages[-1]))

You are standing in front of the haunted mansion. 
The mansion is old and decrepit, with ivy growing up the walls and the windows boarded up. 
The front door is slightly ajar, inviting you inside. 

Sunddenly you hear a loud crash from inside the mansion.

skill: sanity
difficulty: easy


{'sanity': 10, 'name': 'Jone Doe', 'age': '32', 'birthplace': 'Arkham', 'residence': 'New York', 'occupation': 'Journalist', 'backstory': 'John Doe is a journalist who has been investigating the mysterious disappearances in Arkham.\nHe is a skilled investigator and has a knack for finding the truth behind the lies.\n'}
{'sanity': 10, 'name': 'Jone Doe', 'age': '32', 'birthplace': 'Arkham', 'residence': 'New York', 'occupation': 'Journalist', 'backstory': 'John Doe is a journalist who has been investigating the mysterious disappearances in Arkham.\nHe is a skilled investigator and has a knack for finding the truth behind the lies.\n', 'sanity_lost': 1, 'tags': []}
I am making a SANITY check against my own sanity points: 10.
And... FAILED! I lost 1 sanity points.
TURN: 1 


As you fail your sanity check, a wave of unexplainable fear washes over you. For a moment, the world seems to twist and warp. The shadows cast by the setting sun stretch towards you like dark fingers, and the air feel

{'sanity': 10, 'name': 'Jone Doe', 'age': '32', 'birthplace': 'Arkham', 'residence': 'New York', 'occupation': 'Journalist', 'backstory': 'John Doe is a journalist who has been investigating the mysterious disappearances in Arkham.\nHe is a skilled investigator and has a knack for finding the truth behind the lies.\n', 'sanity_lost': 1, 'tags': []}
I select option 1. TURN: 2 


You gather your courage and step through the front door into the mansion. The door creaks loudly as it swings open, echoing through the empty halls. Inside, the air is musty, filled with the scent of decay and old wood. You find yourself in a grand entrance hall, dust covering the floor and a grand staircase leading to the upper levels. 

The mansion seems to be deserted, but there's an unsettling feeling in the air, as if the walls themselves are watching you. As you take a few steps forward, you hear the sound of footsteps upstairs, but when you look, there's nothing there.

Suddenly, you remember the trigger 

{'sanity': 10, 'name': 'Jone Doe', 'age': '32', 'birthplace': 'Arkham', 'residence': 'New York', 'occupation': 'Journalist', 'backstory': 'John Doe is a journalist who has been investigating the mysterious disappearances in Arkham.\nHe is a skilled investigator and has a knack for finding the truth behind the lies.\n', 'sanity_lost': 1, 'tags': []}
I select option 1. TURN: 3 


You decide to head directly to the dining room, guided by a mix of curiosity and the unsettling feeling that whatever resides in this mansion wants you to find it. As you navigate through the corridors, the mansion's eerie silence is oppressive, broken only by the sound of your footsteps and the occasional creak of the old floorboards under your weight.

Finally, you find the dining room. The door is slightly ajar. Pushing it open, you step inside. The room is dominated by a long, dusty table, surrounded by chairs. Faded portraits hang on the walls, and a large, unlit chandelier looms overhead. 

As you take in 

{'sanity': 10, 'name': 'Jone Doe', 'age': '32', 'birthplace': 'Arkham', 'residence': 'New York', 'occupation': 'Journalist', 'backstory': 'John Doe is a journalist who has been investigating the mysterious disappearances in Arkham.\nHe is a skilled investigator and has a knack for finding the truth behind the lies.\n', 'sanity_lost': 1, 'tags': []}
{'sanity': 10, 'name': 'Jone Doe', 'age': '32', 'birthplace': 'Arkham', 'residence': 'New York', 'occupation': 'Journalist', 'backstory': 'John Doe is a journalist who has been investigating the mysterious disappearances in Arkham.\nHe is a skilled investigator and has a knack for finding the truth behind the lies.\n', 'sanity_lost': 7, 'tags': ['insane']}
I am making a SANITY check against my own sanity points: 9.
And... FAILED! I lost 6 sanity points.
I am shocked and lose control of my actions.
TURN: 4  (I'm insane!)


As you fail the sanity check, the reality around you bends and twists in horrifying ways. The walls of the dining room se

{'sanity': 10, 'name': 'Jone Doe', 'age': '32', 'birthplace': 'Arkham', 'residence': 'New York', 'occupation': 'Journalist', 'backstory': 'John Doe is a journalist who has been investigating the mysterious disappearances in Arkham.\nHe is a skilled investigator and has a knack for finding the truth behind the lies.\n', 'sanity_lost': 7, 'tags': ['insane']}
I select option 1. TURN: 5  (I'm insane!)


In your state of insanity, you attempt to communicate with the monstrous creature, your voice trembling as you speak into the thick, oppressive air of the dining room. The creature, a manifestation of your fractured mind, turns its fiery gaze upon you, its form flickering between that of the shadowy dog and something far more sinister.

It responds, not with words, but with a deep, resonant growl that vibrates through the room, shaking the very foundations of the mansion. The portraits on the walls scream in chorus, a cacophony of terror and madness that threatens to overwhelm you.

Yet, am

{'sanity': 10, 'name': 'Jone Doe', 'age': '32', 'birthplace': 'Arkham', 'residence': 'New York', 'occupation': 'Journalist', 'backstory': 'John Doe is a journalist who has been investigating the mysterious disappearances in Arkham.\nHe is a skilled investigator and has a knack for finding the truth behind the lies.\n', 'sanity_lost': 1, 'tags': []}
I select option 1. TURN: 2 


You push the front door open, its hinges groaning with disuse, and step into the foyer of the mansion. The air inside is musty, filled with the scent of decay and old wood. Your footsteps echo on the hardwood floor, and dust motes dance in the shafts of light piercing through the boarded-up windows. Ahead of you is a grand staircase leading to the upper floors, and doors lead off to the left and right into the darkness of the mansion.

As you take a moment to decide your next move, you hear the sound of something scurrying in the walls, and the faint sound of a piano playing somewhere deep within the mansion.

D